In [1]:
import pandas as pd
from datetime import datetime
from pckgs.helper import EpochLogger
from pckgs.headline_preprocess import HeadlinePreprocess
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pandarallel import pandarallel
import matplotlib.pyplot as plt
import seaborn as sb
%load_ext autoreload
%autoreload 2

# import logging
# import os
# logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))

In [2]:
#PREPROCESS
df = pd.read_csv('../datasets/223k crypto news headlines. Dataset. BDCenter Digital.csv', header=0)
df = HeadlinePreprocess.preprocess(df, bigrams=True, tokenize=True)
df.head()

KeyboardInterrupt: 

In [ ]:
#see number of headlines per day
import plotly.express as px

temp = pd.read_csv('../datasets/223k crypto news headlines. Dataset. BDCenter Digital.csv', header=0)
temp.rename(columns = {'publishdate':'date'}, inplace=True)
temp.rename(columns = {'headlinetext':'text'}, inplace=True)
temp.date = temp.date.map(lambda p: datetime.strptime(str(p), '%Y%m%d'))
temp = temp.groupby('date').count()
print(temp.describe())
fig = px.bar(temp, x=temp.index, y='text', color='text')
fig.show()
# plt.figure()
# sb.barplot(x = temp.index, y=temp.text)
del temp

In [3]:
#build tagged documents and assign pandas id as doc id
documents = [TaggedDocument(words = df.loc[i,'text'], tags=[df.index[i]]) for i in range(len(df))]

epoch_logger = EpochLogger(documents)
model = Doc2Vec(dm=1,
                min_count=20,
                window=2,
                vector_size=300,
                workers=3,
                epochs=25,
                seed=23)
                # ,callbacks=[epoch_logger]

#build vocabulary from documents
model.build_vocab(documents, progress_per=10000)

#train model
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

In [2]:
#load model
# temp_path = get_tmpfile('toy_d2v')
# model = Doc2Vec.load(temp_path)
#save model
# model.save('vectors.kv')
# model = Doc2Vec.load('vectors.kv')

In [4]:
#infer vector from every headline
pandarallel.initialize(progress_bar=True)
df['vec'] = df.parallel_apply(lambda row: model.infer_vector(documents[row.name][0], steps=20), axis=1)
#turn column of lists to columns (explode)
df = pd.concat([pd.DataFrame(df['vec'].values.tolist()),df['date']],axis=1)
#set data as index
df.set_index('date', inplace=True, drop=True)
df.head()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
date,,,,,,,,,,,,,,,,,,,,,
2017-07-07,0.045714,-0.001961,0.025097,0.050116,0.124880,0.012416,-0.045367,0.011423,-0.063097,-0.038239,...,0.047621,-0.016316,-0.010944,-0.078619,0.008706,0.068910,0.028491,0.045232,-0.024050,0.012990
2017-10-16,0.056051,-0.048256,0.014275,0.062706,0.213529,-0.009554,-0.040246,0.040103,-0.138210,-0.035579,...,0.057042,-0.011220,-0.074611,-0.137240,0.069290,0.134882,0.023557,0.088169,-0.077396,-0.003851
2017-11-16,0.008569,0.019059,0.023138,0.044631,0.078888,0.010391,-0.019623,0.029473,-0.034051,-0.047822,...,0.002454,-0.043175,-0.017312,-0.052432,-0.012902,0.017311,-0.023603,0.074907,0.008695,0.047033
2017-11-25,0.062763,-0.054391,0.026533,0.091086,0.180778,-0.015789,-0.040906,0.025479,-0.018107,-0.039777,...,0.056327,0.000679,-0.038278,-0.007993,0.040597,-0.000809,-0.002990,0.077777,-0.034933,-0.075832
2018-01-15,-0.007286,0.000952,0.045509,0.031749,0.093047,0.015276,-0.007224,0.001105,-0.010089,-0.057663,...,0.014419,-0.066990,-0.027252,-0.046227,-0.042189,0.029861,-0.007709,0.104993,0.013280,0.017047


In [34]:
#read/write to csv

# df.to_csv('./datasets/headline_vectors.csv')
# df = pd.read_csv('./datasets/headline_vectors.csv', index_col='date', parse_dates=['date'])

In [110]:
example=60000
tmp = documents[example][0]
vector = model.infer_vector(tmp)
most_sim = model.docvecs.most_similar([vector], topn=5)
for (sent, score) in [(documents[most_sim[i][0]].words, most_sim[i][1]) for i in range(len(most_sim))]: print(sent, score)

['huawei', 'to', 'offer', 'first', 'crypto', 'wallet', 'app', 'on', 'latest', 'smartphones'] 0.8514715433120728
['coinjar', 'first', 'bitcoin', 'app', 'to', 'reemerge', 'on', 'iphone', 'app_store'] 0.6404440999031067
['tesla', 'will_be', 'the', 'next', 'amazon', 'unless', 'it', 'gets', 'acquired_by', 'apple', 'first'] 0.5720335245132446
['opera', 'to', 'launch', 'first', 'browser', 'with', 'native', 'crypto', 'wallet'] 0.5637149214744568
['usdx', 'wallet', 'announces', 'integration', 'with', 'first', 'crypto', 'exchange', 'exmarkets'] 0.5604363679885864


In [81]:
example1=['bitcoin', 'is','a','good','investment']
print('Example 1: ' + str(example1))
vector = model.infer_vector(example1, steps=10)
most_sim = model.docvecs.most_similar([vector], topn=5)
print(most_sim)

Example 1: ['bitcoin', 'is', 'a', 'good', 'investment']
[(10496, 0.868197500705719), (3277, 0.8647539019584656), (9353, 0.8615254163742065), (3189, 0.8604573011398315), (2082, 0.8598318099975586)]
